In [ ]:
import os
import sys
import pandas as pd
import shutil
from pynxtools_apm._version import version as pynx_apm_version
from pynxtools_apm.examples.get_sha256_of_directories import SEPARATOR
from pynxtools_apm.examples.oasisb_utils import APT_MIME_TYPES, CAMECA_ROOT_MIME_TYPES
from pynxtools_apm.examples.get_file_from_archive_formats import (
    get_file_from_zip,
    get_file_from_tar,
    get_file_from_rar,
    get_file_from_sevenzip,
)
from pynxtools_apm.examples.oasisb_utils import generate_file_to_hash

# TODO parameterize
with open("source_directory.txt") as fp:
    src_directory = f"{fp.read().strip().replace('/', os.sep)}"
print(src_directory)
with open("target_directory.txt") as fp:
    trg_directory = f"{fp.read().strip().replace('/', os.sep)}"
print(trg_directory)

header = "file_path:archive_path;byte_size;unix_mtime;sha256sum"

## Decompress the original files from the scientists from the storage location

Locally, original research data are stored compressed when not needed.<br>
Maybe multiple compressed files per project directory.<br>

In [ ]:
config: dict[str, str] = {
    "python_version": f"{sys.version.replace(' ', '_')}",
    "working_directory": f"{os.getcwd()}",
    "pynxtools_apm version": f"{pynx_apm_version}",
    # "directory": f"src_directory,  # sys.argv[1],
}

spread_sheet_of_all_projects = pd.read_excel(
    f"{src_directory}{os.sep}aaa_legacy_data.ods",
    sheet_name="aaa_legacy_data",
    engine="odf",
)

# keep_searching_toggle = True
for row in spread_sheet_of_all_projects.itertuples(index=True):
    # process after iterating and ignoring first until projects to process are found
    # if row.project_name not in ("usa_cameca_ulfig"):
    #     if keep_searching_toggle:
    #         continue
    # else:
    #     if keep_searching_toggle:
    #         keep_searching_toggle = False
    # process only one specific project
    if row.project_name not in ("deu_stuttgart_eich_tap"):
        continue
    if row.parse in (1, 2):
        print(f"project{SEPARATOR}{row.project_name}{SEPARATOR}decompress...")

        file_to_hash = generate_file_to_hash(
            f"{src_directory}{os.sep}{row.project_name}.sha256.results.csv"
        )
        spread_sheet_for_project = pd.read_excel(
            f"{src_directory}{os.sep}{row.project_name}.ods",
            sheet_name=row.project_name,
            engine="odf",
        ).fillna("")

        # assume ods is the ground truth configuration with which to work from now on
        for row_idx in range(
            spread_sheet_for_project.shape[0]
        ):  # TODO ?? is the header skipped ?
            for col_idx in range(
                0, 6
            ):  # TODO !!  df.shape[1]), for deu_stuttgart_eich_tap processed further columns if their header not startswith("ignore")
                value = spread_sheet_for_project.iat[row_idx, col_idx]
                if value == "":
                    continue
                else:
                    # trg_directory is a global one
                    trg_file_name = f"{row.project_name}.{row_idx}.{col_idx}.{file_to_hash[value]}.{value[value.rfind('.') + 1 :].lower()}"
                    if value.count(":") == 1:
                        tmp = value.split(":")
                        archiv_file_path = (
                            f"{src_directory}{os.sep}{row.project_name}{os.sep}{tmp[0]}"
                        )
                        file_path = tmp[1]
                        if archiv_file_path.lower().endswith((".zip", ".eln")):
                            if not get_file_from_zip(
                                archiv_file_path,
                                file_path,
                                trg_directory,
                                trg_file_name,
                            ):
                                print(
                                    f"zip >>>> {archiv_file_path}, {file_path} >>>> {trg_file_name}"
                                )
                        elif archiv_file_path.lower().endswith(
                            (".tar", ".tar.gz", ".tar.bz2", ".tar.xz")
                        ):
                            if not get_file_from_tar(
                                archiv_file_path,
                                file_path,
                                trg_directory,
                                trg_file_name,
                            ):
                                print(
                                    f"tar >>>> {archiv_file_path}, {file_path} >>>> {trg_file_name}"
                                )
                        elif archiv_file_path.lower().endswith((".rar")):
                            if not get_file_from_rar(
                                archiv_file_path,
                                file_path,
                                trg_directory,
                                trg_file_name,
                            ):
                                print(
                                    f"rar >>>> {archiv_file_path}, {file_path} >>>> {trg_file_name}"
                                )
                        elif archiv_file_path.lower().endswith((".7z")):
                            if not get_file_from_sevenz(
                                archiv_file_path,
                                file_path,
                                trg_directory,
                                trg_file_name,
                            ):
                                print(
                                    f"sevenz >>>> {archiv_file_path}, {file_path} >>>> {trg_file_name}"
                                )
                        del tmp, archiv_file_path, file_path
                    else:
                        try:
                            shutil.copy2(
                                f"{src_directory}{os.sep}{data}{os.sep}{row.project_name}{value}",
                                f"{trg_directory}{os.sep}{trg_file_name}",
                            )
                        except IOError:
                            print(
                                f"file >>>> {src_directory}{os.sep}{row.project_name}{os.sep}{value} >>>> {trg_file_name}"
                            )
                    del trg_file_name
        del spread_sheet_for_project